In [78]:
import numpy as np
import pandas as pd

from copy import deepcopy
from sklearn.preprocessing import OneHotEncoder

In [79]:
from src.utils import deserialize_data

X_train = deserialize_data("data/interim/X_train.pkl")
y_train = deserialize_data("data/interim/y_train.pkl")

X_valid = deserialize_data("data/interim/X_valid.pkl")
y_valid = deserialize_data("data/interim/y_valid.pkl")

X_test = deserialize_data("data/interim/X_test.pkl")
y_test = deserialize_data("data/interim/y_test.pkl")

In [80]:
X_test

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
26024,31,70550,MORTGAGE,9.0,MEDICAL,F,18000,19.74,0.22,Y,10
6126,24,20000,RENT,8.0,VENTURE,A,6000,7.14,0.30,N,2
20835,29,42000,RENT,0.0,HOMEIMPROVEMENT,D,5000,14.96,0.12,N,9
12036,22,72644,MORTGAGE,3.0,PERSONAL,A,5900,7.49,0.08,N,3
29681,45,35000,OTHER,6.0,PERSONAL,B,4800,NaN,0.14,N,12
...,...,...,...,...,...,...,...,...,...,...,...
2145,24,48000,RENT,0.0,PERSONAL,C,2400,12.99,0.05,Y,4
23920,28,67386,MORTGAGE,11.0,HOMEIMPROVEMENT,A,1500,6.91,0.02,N,6
26739,34,110000,MORTGAGE,15.0,EDUCATION,B,6000,9.45,0.05,N,6
26939,29,120000,MORTGAGE,12.0,VENTURE,B,12000,10.65,0.10,N,9


In [81]:
def drop_duplicate_data(X, y):
    """
    This function drops duplicated data from row X and y.
    
    Parameters
    -----------
    X : dataframe
        features of dataset
        
    y : series
        target of dataset

    Returns
    -------
    X : dataframe
        dropped duplicated data features of dataset
        
    y : dataframe
        dropped duplicated data target of dataset
    """
    
    if not isinstance(X, pd.DataFrame):
        raise TypeError("Fungsi median_imputation: parameter X haruslah bertipe DataFrame!")
    
    if not isinstance(y, pd.Series):
        raise TypeError("Fungsi median_imputation: parameter y haruslah bertipe DataFrame!")
    
    print(f"Fungis drop_duplicate_data telah divalidasi.")
    
    X = X.copy()
    y = y.copy()
    print(f"Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah {X.shape}.")
    
    X_duplicate = X_train[X_train.duplicated()]
    print(f"Fungsi drop_duplicate_data: shape dari data yang duplicate adalah {X_duplicate.shape}.")

    X_clean = (X.shape[0] - X_duplicate.shape[0], X.shape[1])
    print(f"Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah {X_clean}.")
    
    X.drop_duplicates(inplace=True)
    y = y[X.index]
    
    print(f"Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah {X.shape}.")
    
    return X, y

In [82]:
X_train, y_train = drop_duplicate_data(X_train, y_train)

Fungis drop_duplicate_data telah divalidasi.
Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah (26064, 11).
Fungsi drop_duplicate_data: shape dari data yang duplicate adalah (118, 11).
Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah (25946, 11).
Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah (25946, 11).


In [83]:
def median_imputation(data, subset_data, fit):
    """
    Parameters
    -----------
    data : dataframe
        dataset to be imputed
        
    subset_data : list of string
        columns name

    fit : boolean
        if fit=true, this function will return median of subset_data
        if fit=false, this function will impute the data based on subset_data

    Returns
    -------
    X : dataframe
        dropped duplicated data features of dataset
        
    y : dataframe
        dropped duplicated data target of dataset
    """
    
    if not isinstance(data, pd.DataFrame):
        raise TypeError("Fungsi median_imputation: parameter data haruslah bertipe DataFrame!")
    
    if fit is True and not isinstance(subset_data, list):
        raise TypeError(
            "Fungsi median_imputation: untuk nilai parameter fit = True, subset_data harus bertipe list dan berisi " 
            "daftar nama kolom yang ingin dicari nilai mediannya guna menjadi data imputasi pada kolom tersebut")
    
    if fit is False and not isinstance(subset_data, dict):
        raise TypeError(
            "Fungsi median_imputation: untuk nilai parameter fit = False, subset_data harus bertipe dict dan berisi "
            "key yang merupakan nama kolom beserta value yang merupakan nilai median dari kolom tersebut")
    
    if not isinstance(fit, bool):
        raise TypeError("Fungsi median_imputation: parameter fit haruslah bertipe boolean, bernilai True atau False.")
    
    print("Fungsi median_imputation: parameter telah divalidasi.")
    
    data = data.copy()
    subset_data = deepcopy(subset_data) 
    
    """
    Handles fitting data
    """
    if fit is True:
        imputation_data = {}
        for subset in subset_data:
            imputation_data[subset] = data[subset].median(numeric_only=True)
            
        print(f"Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {imputation_data}")
            
        return imputation_data
    
    """
    Handles transforming data
    """
    print("Fungsi median_imputation: informasi count na sebelum dilakukan imputasi")
    print(data.isna().sum())
    print()
    
    for subset in subset_data:
        data[subset] = data[subset].fillna(subset_data[subset])

    print("Fungsi median_imputation: informasi count na setelah dilakukan imputasi.")
    print(data.isna().sum())
    print()
    
    return data

In [84]:
X_train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
32377,64,46000,RENT,2.0,PERSONAL,C,4800,11.09,0.10,Y,24
1338,26,26000,OWN,0.0,DEBTCONSOLIDATION,E,8500,16.45,0.33,N,3
7047,23,51000,MORTGAGE,3.0,PERSONAL,C,16000,13.11,0.31,Y,3
8225,22,56004,MORTGAGE,6.0,MEDICAL,A,6000,7.88,0.11,N,4
7178,24,79000,RENT,3.0,PERSONAL,C,7000,12.54,0.09,N,3


In [85]:
subset_data = ["person_age", "person_income", "person_emp_length", "loan_amnt", "loan_int_rate", "loan_percent_income", "cb_person_cred_hist_length"]
subset_data = median_imputation(X_train, subset_data, fit=True)
X_train = median_imputation(X_train, subset_data, fit=False)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {'person_age': np.float64(26.0), 'person_income': np.float64(55000.0), 'person_emp_length': np.float64(4.0), 'loan_amnt': np.float64(8000.0), 'loan_int_rate': np.float64(10.99), 'loan_percent_income': np.float64(0.15), 'cb_person_cred_hist_length': np.float64(4.0)}
Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              711
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 2463
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi.
person_ag

In [86]:
X_train

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
32377,64,46000,RENT,2.0,PERSONAL,C,4800,11.09,0.10,Y,24
1338,26,26000,OWN,0.0,DEBTCONSOLIDATION,E,8500,16.45,0.33,N,3
7047,23,51000,MORTGAGE,3.0,PERSONAL,C,16000,13.11,0.31,Y,3
8225,22,56004,MORTGAGE,6.0,MEDICAL,A,6000,7.88,0.11,N,4
7178,24,79000,RENT,3.0,PERSONAL,C,7000,12.54,0.09,N,3
...,...,...,...,...,...,...,...,...,...,...,...
29802,39,38500,MORTGAGE,7.0,MEDICAL,C,3500,13.98,0.09,Y,17
5390,25,69000,RENT,5.0,HOMEIMPROVEMENT,A,8500,6.92,0.12,N,4
860,26,148000,RENT,1.0,DEBTCONSOLIDATION,E,20000,17.99,0.14,N,3
15795,26,175000,MORTGAGE,0.0,PERSONAL,C,15000,10.99,0.09,N,3


In [87]:
subset_data = ["person_age", "person_income", "person_emp_length", "loan_amnt", "loan_int_rate", "loan_percent_income", "cb_person_cred_hist_length"]
subset_data = median_imputation(X_valid, subset_data, fit=True)
X_valid = median_imputation(X_valid, subset_data, fit=False)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {'person_age': np.float64(26.0), 'person_income': np.float64(55992.0), 'person_emp_length': np.float64(4.0), 'loan_amnt': np.float64(8000.0), 'loan_int_rate': np.float64(10.99), 'loan_percent_income': np.float64(0.15), 'cb_person_cred_hist_length': np.float64(4.0)}
Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi
person_age                      0
person_income                   0
person_home_ownership           0
person_emp_length             146
loan_intent                     0
loan_grade                      0
loan_amnt                       0
loan_int_rate                 513
loan_percent_income             0
cb_person_default_on_file       0
cb_person_cred_hist_length      0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi.
person_age          

In [88]:
subset_data = ["person_age", "person_income", "person_emp_length", "loan_amnt", "loan_int_rate", "loan_percent_income", "cb_person_cred_hist_length"]
subset_data = median_imputation(X_test, subset_data, fit=True)
X_test = median_imputation(X_test, subset_data, fit=False)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {'person_age': np.float64(26.0), 'person_income': np.float64(54000.0), 'person_emp_length': np.float64(4.0), 'loan_amnt': np.float64(7600.0), 'loan_int_rate': np.float64(11.14), 'loan_percent_income': np.float64(0.15), 'cb_person_cred_hist_length': np.float64(4.0)}
Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi
person_age                      0
person_income                   0
person_home_ownership           0
person_emp_length              32
loan_intent                     0
loan_grade                      0
loan_amnt                       0
loan_int_rate                 124
loan_percent_income             0
cb_person_default_on_file       0
cb_person_cred_hist_length      0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi.
person_age          

In [89]:
X_test

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
26024,31,70550,MORTGAGE,9.0,MEDICAL,F,18000,19.74,0.22,Y,10
6126,24,20000,RENT,8.0,VENTURE,A,6000,7.14,0.30,N,2
20835,29,42000,RENT,0.0,HOMEIMPROVEMENT,D,5000,14.96,0.12,N,9
12036,22,72644,MORTGAGE,3.0,PERSONAL,A,5900,7.49,0.08,N,3
29681,45,35000,OTHER,6.0,PERSONAL,B,4800,11.14,0.14,N,12
...,...,...,...,...,...,...,...,...,...,...,...
2145,24,48000,RENT,0.0,PERSONAL,C,2400,12.99,0.05,Y,4
23920,28,67386,MORTGAGE,11.0,HOMEIMPROVEMENT,A,1500,6.91,0.02,N,6
26739,34,110000,MORTGAGE,15.0,EDUCATION,B,6000,9.45,0.05,N,6
26939,29,120000,MORTGAGE,12.0,VENTURE,B,12000,10.65,0.10,N,9


In [59]:
X_test

32198    0
6319     0
32482    0
27255    0
6148     0
        ..
22942    0
17882    0
2955     0
26768    0
24085    0
Name: loan_status, Length: 5213, dtype: int64